Concorrência e paralelismo

In [ ]:
>>> import time
>>> import random
>>> from threading import Thread
>>> class AvisarSistema(Thread):
...     def __init__ (self, url_do_sistema):
...         Thread.__init__(self)
...         self.url_to_call = url_do_sistema
...     def run(self):
...         # define um valor aleatório entre 0.5 e 2.5
...         sleep_time = random.uniform(0.5, 2.5)
...         # suspende a thread pelo tempo definido (em segundos)
...         time.sleep(sleep_time)
...         print(f">>> Calling {self.url_to_call}")
>>> a = AvisarSistema("Url do sistema A")
>>> a.start()
>>> b = AvisarSistema("Url do sistema B")
>>> b.start()
>>> c = AvisarSistema("Url do sistema C")
>>> c.start()

GIL — Global Interpreter Lock

In [1]:
import time
import queue
from multiprocessing import Process, Queue, current_process

def processar_fila_de_pedidos(novos_pedidos, pedidos_processados):
    '''
    A função espera receber dois objetos do tipo Queue
    '''
    while True:
        try:
            '''
            pega um pedido da fila, a função get_nowait() levantará
            uma queue.Empty exception se a fila estiver vazia
            '''
            pedido = novos_pedidos.get_nowait()
        except queue.Empty:
            break
        else:
            # Processa o pedido e adiciona a fila de pedidos processados
            print(pedido[1])
            pedidos_processados.put(
                f'Pedido número: {pedido[0]}, ' + \
                'foi processado por {current_process().name}'
            )
            time.sleep(.5)
    return True

def processar_pedidos():
    quantidade_de_pedidos = 10
    quantidade_de_processos = 4
    novos_pedidos = Queue()
    pedidos_processados = Queue()
    processos = []
    for i in range(quantidade_de_pedidos):
        # para nosso exemplo, um pedido é uma tupla com id, descricao
        pedido = (i, f"Pedido de compra número: {i}")
        novos_pedidos.put(pedido)
    # criando os processos
    for w in range(quantidade_de_processos):
        '''
        O parametro TARGET recebe a funcão que será executada e o
        parametro ARGS os paramentros da função passada no parametro
        anterior.
        '''
        p = Process(
            target=processar_fila_de_pedidos,
            args=(novos_pedidos, pedidos_processados)
        )
        processos.append(p)
        p.start()
    # finalizando os processos
    for p in processos:
        p.join()
    # imprime os resultados
    while not pedidos_processados.empty():
        print(pedidos_processados.get())
    return True

if __name__ == '__main__':
    processar_pedidos()

Futures e Objetos de espera (awaitable objects)

In [ ]:
>>> import asyncio
>>> # Executa uma atividade até ser cancelada
>>> async def rotina_rapida():
...     print("Executando rotina_rapida")
...     i = 1
...     while True:
...         print('.' * i)
...         # Aguarda o retorno da função sleep(1).
...         # Nesse momento o event loop entende
...         # que a mudança de contexto pode acontecer.
...         await asyncio.sleep(1)
...         i += 1
>>> # Executa uma atividade demorada
>>> async def rotina_muito_lenta():
...     print("Executando rotina_muito_lenta...")
...     # Aguarda o retorno da função sleep(10)
...     # Nesse momento o event loop entende
...     # que a mudança de contexto pode acontecer.
...     await asyncio.sleep(10)
...     print("rotina_muito_lenta finalizada!")
>>> # Executa as rotinas
>>> async def executar_rotinas():
...     # Cria a Task para controle da tarefa rápida
...     tarefa_rapida = asyncio.ensure_future(rotina_rapida())
...     # Dispara a execução da rotina muito lenta
...     await rotina_muito_lenta()
...     # Cancela a execução da tarefa rápida
...     tarefa_rapida.cancel()
>>> # Pega o loop de eventos
>>> loop = asyncio.get_event_loop()
>>> # Executa a rotina até finalizar todas as instruções
>>> loop.run_until_complete(executar_rotinas())
>>> # Encerra o loop de eventos
>>> loop.close()

Pontos de atenção na programação assíncrona

In [ ]:
>>> async def get_content(url='http://www.Google.com'):
...     response = requests.get(url)
...     return response

In [ ]:
>>> async def get_content(url='http://www.Google.com')
...     response = await aiohttp.request('GET', url)
...     return response